In [1]:
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data,DataLoader
from functions import *
from pytorch_util import *
from torch.optim import Adam
from torch.nn.utils import clip_grad_value_

In [2]:
with open('../Data/train_data_ACSF.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('../Data/val_data_ACSF.pickle', 'rb') as handle:
    val_data = pickle.load(handle)
with open('../Data/test_data_ACSF.pickle', 'rb') as handle:
    test_data = pickle.load(handle)

In [3]:
### parameters ###
batch_size = 32
dim = 64
edge_dim = 12
epochs = 5
clip = 2

### load dataset

In [4]:
train_list = [Data(**d) for d in train_data]
train_dl = DataLoader(train_list,batch_size,shuffle=True)

In [5]:
val_list = [Data(**d) for d in val_data]
valid_dl = DataLoader(val_list,batch_size,shuffle=False)

In [6]:
test_list = [Data(**d) for d in test_data]
test_dl = DataLoader(test_list,batch_size,shuffle=False)

In [20]:
train_list[0]

Data(edge_attr=[36, 19], edge_attr3=[58, 8], edge_attr4=[58, 1], edge_index=[2, 36], edge_index3=[2, 58], x=[17, 89], y=[58])

In [8]:
# change layers for bonding from 2 to 3
class Net_int_2Edges(torch.nn.Module):
    # use both types of edges
    def __init__(self,dim=64,edge_dim=12,node_in=8,edge_in=19,edge_in3=8):
        super(Net_int_2Edges, self).__init__()
        self.lin_node = torch.nn.Linear(node_in, dim)
        self.lin_edge_attr = torch.nn.Linear(edge_in, edge_dim)
        
        nn1 = Linear(edge_dim, dim * dim, bias=False)
        nn2 = Linear(edge_in3, dim * dim * 2 * 2, bias=False)
        
        self.conv1 = NNConv(dim, dim, nn1, aggr='mean', root_weight=False)
        self.gru1 = GRU(dim, dim)
        self.lin_covert = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim*2),RReLU())
        
        self.conv2 = NNConv(dim*2, dim*2, nn2, aggr='mean', root_weight=False)
        self.gru2 = GRU(dim*2, dim*2)
        
        self.lin_weight = Linear(8, dim*3*2, bias=False)
        self.lin_bias = Linear(8, 1, bias=False)
        self.norm = BatchNorm1d(dim*3*2)
        self.norm_x = BatchNorm1d(node_in)
        
    def forward(self, data,IsTrain=False):
        out = F.rrelu(self.lin_node(self.norm_x(data.x)))
        edge_attr = F.rrelu(self.lin_edge_attr(data.edge_attr))
        h = out.unsqueeze(0)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        edge_attr3 = torch.cat([data.edge_attr3,data.edge_attr3],0)
        
        for i in range(3):
            # using bonding as edge
            m = F.rrelu(self.conv1(out, data.edge_index, edge_attr))
            out, h = self.gru1(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.lin_covert(out)
        h = out.unsqueeze(0)
        for i in range(2):
            # using couping as edge
            m = F.rrelu(self.conv2(out, edge_index3, edge_attr3))
            out, h = self.gru2(m.unsqueeze(0), h)
            out = out.squeeze(0)     
            
        temp = out[data.edge_index3] # (2,N,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)
        yhat = self.norm(yhat)
        weight = self.lin_weight(data.edge_attr3)
        bias = self.lin_bias(data.edge_attr3)
        yhat = torch.sum(yhat * weight,1,keepdim=True) + bias
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat    

### build model and set up training

In [9]:
model = Net_int_2Edges(dim=dim,edge_dim=edge_dim,node_in=89).to('cuda:0')

In [10]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=1e-4)

In [11]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:1.1547897267570548, val_loss:0.8259136740468506
epoch:1, train_loss:0.7587013766408021, val_loss:0.5219322294633613
epoch:2, train_loss:0.4495059319920446, val_loss:0.2876564690158663
epoch:3, train_loss:0.330116195947436, val_loss:0.1743541665904574
epoch:4, train_loss:0.26789794926998767, val_loss:0.10732892579120448
Training completed in 189.63176155090332s


In [13]:
# change layers for coupling from 2 to 3
class Net_int_2Edges(torch.nn.Module):
    # use both types of edges
    def __init__(self,dim=64,edge_dim=12,node_in=8,edge_in=19,edge_in3=8):
        super(Net_int_2Edges, self).__init__()
        self.lin_node = torch.nn.Linear(node_in, dim)
        self.lin_edge_attr = torch.nn.Linear(edge_in, edge_dim)
        
        nn1 = Linear(edge_dim, dim * dim, bias=False)
        nn2 = Linear(edge_in3, dim * dim * 2 * 2, bias=False)
        
        self.conv1 = NNConv(dim, dim, nn1, aggr='mean', root_weight=False)
        self.gru1 = GRU(dim, dim)
        self.lin_covert = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim*2),RReLU())
        
        self.conv2 = NNConv(dim*2, dim*2, nn2, aggr='mean', root_weight=False)
        self.gru2 = GRU(dim*2, dim*2)
        
        self.lin_weight = Linear(8, dim*3*2, bias=False)
        self.lin_bias = Linear(8, 1, bias=False)
        self.norm = BatchNorm1d(dim*3*2)
        self.norm_x = BatchNorm1d(node_in)
        
    def forward(self, data,IsTrain=False):
        out = F.rrelu(self.lin_node(self.norm_x(data.x)))
        edge_attr = F.rrelu(self.lin_edge_attr(data.edge_attr))
        h = out.unsqueeze(0)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        edge_attr3 = torch.cat([data.edge_attr3,data.edge_attr3],0)
        
        for i in range(2):
            # using bonding as edge
            m = F.rrelu(self.conv1(out, data.edge_index, edge_attr))
            out, h = self.gru1(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.lin_covert(out)
        h = out.unsqueeze(0)
        for i in range(3):
            # using couping as edge
            m = F.rrelu(self.conv2(out, edge_index3, edge_attr3))
            out, h = self.gru2(m.unsqueeze(0), h)
            out = out.squeeze(0)     
            
        temp = out[data.edge_index3] # (2,N,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)
        yhat = self.norm(yhat)
        weight = self.lin_weight(data.edge_attr3)
        bias = self.lin_bias(data.edge_attr3)
        yhat = torch.sum(yhat * weight,1,keepdim=True) + bias
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat    

In [14]:
model = Net_int_2Edges(dim=dim,edge_dim=edge_dim,node_in=89).to('cuda:0')

In [15]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=1e-4)

In [16]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:1.1632127671357317, val_loss:0.7997429537722188
epoch:1, train_loss:0.7755193334221022, val_loss:0.5500625839345475
epoch:2, train_loss:0.4783027646786337, val_loss:0.2719283985037707
epoch:3, train_loss:0.3302032805478458, val_loss:0.19090469108305425
epoch:4, train_loss:0.2653944992980267, val_loss:0.1330681057790151
Training completed in 207.56409883499146s


In [18]:
# untie weights
class Net_int_2Edges(torch.nn.Module):
    # use both types of edges
    def __init__(self,dim=64,edge_dim=12,node_in=8,edge_in=19,edge_in3=8):
        super(Net_int_2Edges, self).__init__()
        self.lin_node = torch.nn.Linear(node_in, dim)
        self.lin_edge_attr = torch.nn.Linear(edge_in, edge_dim)
        
        self.conv1 = nn.ModuleList([NNConv(dim, dim, Linear(edge_dim, dim * dim, bias=False), \
                                           aggr='mean', root_weight=False) for _ in range(2)])
        self.gru1 = nn.ModuleList([GRU(dim, dim) for _ in range(2)])
        
        self.lin_covert = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim*2),RReLU())
        
        self.conv2 = nn.ModuleList([NNConv(dim*2, dim*2, Linear(edge_in3, dim * dim * 2 * 2, bias=False),\
                                           aggr='mean', root_weight=False) for _ in range(2)])
        self.gru2 = nn.ModuleList([GRU(dim*2, dim*2) for _ in range(2)])
        
        self.lin_weight = Linear(8, dim*3*2, bias=False)
        self.lin_bias = Linear(8, 1, bias=False)
        self.norm = BatchNorm1d(dim*3*2)
        self.norm_x = BatchNorm1d(node_in)
        
    def forward(self, data,IsTrain=False):
        out = F.rrelu(self.lin_node(self.norm_x(data.x)))
        edge_attr = F.rrelu(self.lin_edge_attr(data.edge_attr))
        h = out.unsqueeze(0)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        edge_attr3 = torch.cat([data.edge_attr3,data.edge_attr3],0)
        
        for conv,gru in zip(self.conv1,self.gru1):
            # using bonding as edge
            m = F.rrelu(conv(out, data.edge_index, edge_attr))
            out, h = gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.lin_covert(out)
        h = out.unsqueeze(0)
        
        for conv,gru in zip(self.conv2,self.gru2):
            # using couping as edge
            m = F.rrelu(conv(out, edge_index3, edge_attr3))
            out, h = gru(m.unsqueeze(0), h)
            out = out.squeeze(0)     
            
        temp = out[data.edge_index3] # (2,N,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)
        yhat = self.norm(yhat)
        weight = self.lin_weight(data.edge_attr3)
        bias = self.lin_bias(data.edge_attr3)
        yhat = torch.sum(yhat * weight,1,keepdim=True) + bias
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat    

In [19]:
model = Net_int_2Edges(dim=dim,edge_dim=edge_dim,node_in=89).to('cuda:0')

In [20]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=1e-4)

In [21]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:1.1594490586321378, val_loss:0.8086355328559875
epoch:1, train_loss:0.7355345898177126, val_loss:0.5043860161279001
epoch:2, train_loss:0.42348383854228067, val_loss:0.18682799665018535
epoch:3, train_loss:0.2614997324568254, val_loss:0.1339515131762904
epoch:4, train_loss:0.19197627482999663, val_loss:0.034456392805864156
Training completed in 169.59033465385437s


In [22]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.13746430281980648, val_loss:-0.002662224894087029
epoch:1, train_loss:0.0945514630480313, val_loss:-0.054346995969486035
epoch:2, train_loss:0.053573113426252705, val_loss:-0.0722985656804636
epoch:3, train_loss:0.01870357307976356, val_loss:-0.10433403677187669
epoch:4, train_loss:-0.014622645656172413, val_loss:-0.14452321372894394
Training completed in 169.0499165058136s


In [23]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.0413524946023271, val_loss:-0.1542664094008187
epoch:1, train_loss:-0.065125956247332, val_loss:-0.19085314467899564
epoch:2, train_loss:-0.08885088705878907, val_loss:-0.21685146457619137
epoch:3, train_loss:-0.11049663226763626, val_loss:-0.24656966812590247
epoch:4, train_loss:-0.1302875803692405, val_loss:-0.2716748599185903
Training completed in 169.9624707698822s


In [24]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.15095552611988758, val_loss:-0.28606100918518174
epoch:1, train_loss:-0.16858280100572257, val_loss:-0.30791402555620057
epoch:2, train_loss:-0.1859920694248748, val_loss:-0.300036578049135
epoch:3, train_loss:-0.20444482100755318, val_loss:-0.33359170318222964
epoch:4, train_loss:-0.22066242348319273, val_loss:-0.33999692607257104
Training completed in 169.8011360168457s


In [25]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.2333877906782419, val_loss:-0.37278015211097193
epoch:1, train_loss:-0.24728182025652362, val_loss:-0.3677361056718052
epoch:2, train_loss:-0.2631969788219485, val_loss:-0.3892742553009437
epoch:3, train_loss:-0.2746893775942776, val_loss:-0.403559311460226
epoch:4, train_loss:-0.28780888158965995, val_loss:-0.43840433653985333
Training completed in 171.43177342414856s


In [26]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.29909109896892927, val_loss:-0.4306042927962083
epoch:1, train_loss:-0.3090799196048801, val_loss:-0.4241817949546708
epoch:2, train_loss:-0.3196433461354266, val_loss:-0.4257182358867592
epoch:3, train_loss:-0.3294254029293174, val_loss:-0.47497534204242575
epoch:4, train_loss:-0.34052170920055863, val_loss:-0.44629317541153
Training completed in 170.09883213043213s


In [27]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.3496275109396059, val_loss:-0.4916843884011619
epoch:1, train_loss:-0.3603542146273383, val_loss:-0.47067818943506634
epoch:2, train_loss:-0.3672700528963331, val_loss:-0.5043829705598007
epoch:3, train_loss:-0.37823523475418475, val_loss:-0.506532619906287
epoch:4, train_loss:-0.3872226643396067, val_loss:-0.5169145995353022
Training completed in 169.93472170829773s


In [28]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.392837257504763, val_loss:-0.5248468982485625
epoch:1, train_loss:-0.3999615752831899, val_loss:-0.4886773057982453
epoch:2, train_loss:-0.41064669176166224, val_loss:-0.522937386845931
epoch:3, train_loss:-0.41749444948343256, val_loss:-0.5377823637209387
epoch:4, train_loss:-0.4238019130670921, val_loss:-0.5531528641780218
Training completed in 170.36451745033264s


In [29]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.4318233698906286, val_loss:-0.5468512849930005
epoch:1, train_loss:-0.43636129336592594, val_loss:-0.5480451825210172
epoch:2, train_loss:-0.4462606151408129, val_loss:-0.5532766004276072
epoch:3, train_loss:-0.45291176333403443, val_loss:-0.5944341593063794
epoch:4, train_loss:-0.4564936625314293, val_loss:-0.5816825665215142
Training completed in 169.9697141647339s


In [30]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.46464196545896763, val_loss:-0.5822738100830306
epoch:1, train_loss:-0.47128466078964715, val_loss:-0.5983556186159452
epoch:2, train_loss:-0.47720149661587646, val_loss:-0.6123289756922641
epoch:3, train_loss:-0.483131457339916, val_loss:-0.6263906406795877
epoch:4, train_loss:-0.4875799993374166, val_loss:-0.6074130747180718
Training completed in 169.859388589859s


In [31]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.49376451935079146, val_loss:-0.6199501685352407
epoch:1, train_loss:-0.49977215103188444, val_loss:-0.6315819131385567
epoch:2, train_loss:-0.5055402557109664, val_loss:-0.6382716247159191
epoch:3, train_loss:-0.5121168648755109, val_loss:-0.6393220304933369
epoch:4, train_loss:-0.5165831072176762, val_loss:-0.6165233023910441
Training completed in 169.81035041809082s


In [32]:
for para in opt.param_groups:
    para['lr'] = 2e-5

In [33]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.5633440023423549, val_loss:-0.7016189865704275
epoch:1, train_loss:-0.5675841959906213, val_loss:-0.6992784708610966
epoch:2, train_loss:-0.570821271552634, val_loss:-0.7009264338984449
epoch:3, train_loss:-0.5735327497594804, val_loss:-0.7016119856992339
epoch:4, train_loss:-0.5742597658175227, val_loss:-0.7037720715897715
Training completed in 170.8988790512085s


In [45]:
checkpoint = torch.load('../Model/gnn_int_logloss_2Edges.tar')
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

In [34]:
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, '../Model/gnn_int_logloss_2Edges_ACSF2.tar')

### make submissions

In [35]:
model.eval()
yhat_list = []
with torch.no_grad():
    for data in test_dl:
        data = data.to('cuda:0')
        yhat_list.append(model(data,False))

yhat = torch.cat(yhat_list).cpu().detach().numpy()
submission = pd.read_csv('../Data/sample_submission.csv')
submission['scalar_coupling_constant'] = yhat
submission.to_csv('../Submission/gnn_int_logloss_2Edges_ACSF.csv',index=False)